In [2]:
import tensorflow as tf
from keras import layers, models
from keras.datasets import mnist
from keras.utils import to_categorical
import numpy as np

In [3]:
print(tf.config.list_physical_devices('GPU'))
print(tf.__version__)
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2.19.0


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [24]:
# Load dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# reshape the image with to single channel
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# padding to change image size to 32x32
# train_images = np.pad(train_images, ((0,0),(2,2),(2,2),(0,0)), 'constant')
# test_images = np.pad(test_images, ((0,0),(2,2),(2,2),(0,0)), 'constant')

train_images.shape

(60000, 28, 28, 1)

In [25]:
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

# one hot encode labeling
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [26]:
def build_lenet5(input_shape=(28, 28, 1), num_classes=10):
  model = models.Sequential()

  # Conv1
  model.add(layers.Conv2D(6, kernel_size=(5, 5), activation='tanh', input_shape=input_shape, name='C1'))
  # Pool1
  model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), name='S2'))
  # Conv2
  model.add(layers.Conv2D(16, kernel_size=(5, 5), activation='tanh', name='C3'))
  # Pool2
  model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), name='S4'))
  model.add(layers.Flatten(name='Flatten'))
  # FC1
  model.add(layers.Dense(120, activation='tanh', name='C5_FC1'))
  # FC2
  model.add(layers.Dense(84, activation='tanh', name='FC2'))
  # Output
  model.add(layers.Dense(num_classes, activation='softmax', name='Output'))

  return model

In [27]:
model = build_lenet5()
model.summary()

/Users/stefanopark/Documents/FPGA-CNN-Accelerator/.venv/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ C1 (Conv2D)                     │ (None, 24, 24, 6)      │           156 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ S2 (AveragePooling2D)           │ (None, 12, 12, 6)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ C3 (Conv2D)                     │ (None, 8, 8, 16)       │         2,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ S4 (AveragePooling2D)           │ (None, 4, 4, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Flatten (Flatten)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ C5_FC1 (Dense)                  │ (None, 120)            │        30,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ FC2 (Dense)                     │ (None, 84)             │        10,164 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output (Dense)                  │ (None, 10)             │           850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,426 (173.54 KB)

 Trainable params: 44,426 (173.54 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [29]:
test_labels.shape

(10000, 10)

In [30]:
# Train
history = model.fit(train_images, train_labels,
                    epochs=10,
                    batch_size=64,
                    validation_data=(test_images, test_labels))

Epoch 1/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 22s 22ms/step - accuracy: 0.8532 - loss: 0.5102 - val_accuracy: 0.9605 - val_loss: 0.1299
Epoch 2/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 21s 23ms/step - accuracy: 0.9632 - loss: 0.1146 - val_accuracy: 0.9701 - val_loss: 0.0908
Epoch 3/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 21s 22ms/step - accuracy: 0.9780 - loss: 0.0715 - val_accuracy: 0.9789 - val_loss: 0.0639
Epoch 4/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 21s 22ms/step - accuracy: 0.9828 - loss: 0.0525 - val_accuracy: 0.9820 - val_loss: 0.0570
Epoch 5/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 21s 22ms/step - accuracy: 0.9874 - loss: 0.0410 - val_accuracy: 0.9820 - val_loss: 0.0606
Epoch 6/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 21s 22ms/step - accuracy: 0.9896 - loss: 0.0347 - val_accuracy: 0.9840 - val_loss: 0.0541
Epoch 7/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 21s 22ms/step - accuracy: 0.9914 - loss: 0.0276 - val_accuracy: 0.9846 - val_loss: 0.0509
Epoch 8/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 21s 22ms/step - accuracy: 0.9929 - loss: 0.0231 - 

In [31]:
# Eval
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.9823 - loss: 0.0541
Test accuracy: 0.9861999750137329


In [37]:
# Trying Predict
sample_images = test_images[30:50]
sample_labels = np.argmax(test_labels[30:50], axis=1)

predictions = model.predict(sample_images)
predicted_labels = np.argmax(predictions, axis=1)

for i in range(len(sample_images)):
    print(f"Image {i+1}: True Label = {sample_labels[i]}, Predicted Label = {predicted_labels[i]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Image 1: True Label = 3, Predicted Label = 3
Image 2: True Label = 1, Predicted Label = 1
Image 3: True Label = 3, Predicted Label = 3
Image 4: True Label = 4, Predicted Label = 4
Image 5: True Label = 7, Predicted Label = 7
Image 6: True Label = 2, Predicted Label = 2
Image 7: True Label = 7, Predicted Label = 7
Image 8: True Label = 1, Predicted Label = 1
Image 9: True Label = 2, Predicted Label = 2
Image 10: True Label = 1, Predicted Label = 1
Image 11: True Label = 1, Predicted Label = 1
Image 12: True Label = 7, Predicted Label = 7
Image 13: True Label = 4, Predicted Label = 4
Image 14: True Label = 2, Predicted Label = 2
Image 15: True Label = 3, Predicted Label = 3
Image 16: True Label = 5, Predicted Label = 5
Image 17: True Label = 1, Predicted Label = 1
Image 18: True Label = 2, Predicted Label = 2
Image 19: True Label = 4, Predicted Label = 4
Image 20: True Label = 4, Predicted Label = 4


In [38]:
# Work of Gemini

def export_weights_to_header(model, filename="lenet_weights.h"):
    """
    Extracts weights and biases from a Keras model and writes them
    to a C header file.
    """
    print(f"\nExporting weights to {filename}...")
    with open(filename, 'w') as f:
        f.write("#ifndef LENET_WEIGHTS_H\n")
        f.write("#define LENET_WEIGHTS_H\n\n")
        f.write("// This file was generated by an automated Python script.\n")
        f.write("// It contains the weights and biases for the LeNet-5 model.\n\n")

        for layer in model.layers:
            weights_and_biases = layer.get_weights()
            if not weights_and_biases:  # If the layer has no trainable parameters (e.g., Pooling)
                continue

            # Layer name for C variable names
            layer_name = layer.name.replace('-', '_')

            # There are two items: weights and biases
            weights = weights_and_biases[0]
            biases = weights_and_biases[1]

            # --- Write Weights ---
            f.write(f"// Layer: {layer.name} - Weights\n")
            # For Conv layers, weights are (height, width, in_channels, out_channels)
            # For Dense layers, weights are (input_features, output_features)
            # We will flatten them in C order (row-major).
            flat_weights = weights.flatten()
            f.write(f"const float {layer_name}_weights[] = {{\n    ")
            for i, w in enumerate(flat_weights):
                f.write(f"{w:.6f}f, ")
                if (i + 1) % 12 == 0:
                    f.write("\n    ")
            f.write("\n};\n\n")

            # --- Write Biases ---
            f.write(f"// Layer: {layer.name} - Biases\n")
            flat_biases = biases.flatten()
            f.write(f"const float {layer_name}_biases[] = {{\n    ")
            for i, b in enumerate(flat_biases):
                f.write(f"{b:.6f}f, ")
                if (i + 1) % 12 == 0:
                    f.write("\n    ")
            f.write("\n};\n\n")

        f.write("#endif // LENET_WEIGHTS_H\n")
    print(f"Successfully exported {len(model.layers)} layers to {filename}.")

In [39]:
export_weights_to_header(model)


Exporting weights to lenet_weights.h...
Successfully exported 8 layers to lenet_weights.h.
